## Input fiddler client

In [ ]:
import fiddler as fdl
import pandas as pd

client = fdl.FiddlerApi()

If dataset is already in Fiddler, just use it. Otherwise upload it.

In [ ]:
if 'wine_quality' in client.list_datasets():
    df_schema = client.get_dataset_info('wine_quality')
    df = client.get_dataset('wine_quality')['train']
else:
    df = pd.read_csv('/app/fiddler_samples/samples/datasets/winequality/train.csv')
    df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)
    upload_result = client.upload_dataset(
        dataset={'train': df}, 
        dataset_id='wine_quality')

## Create model schema

In [ ]:
target = 'quality'

feature_columns = df_schema.get_column_names()
feature_columns.remove('row_id')
feature_columns.remove(target)

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=df_schema,
    target=target, 
    features=feature_columns,
    display_name='external model',
    description='this is an external model called from fiddler via rest API'
)

# Write model schema

In [ ]:
import pathlib
import shutil
import yaml

model_dir = pathlib.Path('external_model')
shutil.rmtree(model_dir, ignore_errors=True)
model_dir.mkdir()

with open(model_dir / 'model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)


## Write package.py

In [ ]:
%%writefile external_model/package.py

from pathlib import Path
import pandas as pd
import requests
import json
import logging

# make sure model generated in tutorial 02 is deployed to server
external_model_endpoint = 'http://host.docker.internal:5100/execute/onebox/tutorial/wine_quality_model'


class ExternalModelPackage:
    is_classifier = False

    def predict(self, input_df):
        logging.info(f'input df: {input_df}')
        # convert input datafrme to a format that external model accepts
        data_array = [y.iloc[0,:].to_dict() for x , y in input_df.groupby(level=0)]
        data = dict(data=data_array)
        json_input = json.dumps(data)

        # call external service
        json_result = self.invoke_external_model(json_input)
        
        # convert response back to dataframe
        return pd.DataFrame(json_result)
    
    # invoke the externa model using API
    def invoke_external_model(self, json_input):
        logging.info(f'input json: {json_input}')
        headers = {'Content-type': 'application/json'}
        result = requests.post(external_model_endpoint, 
                               headers=headers, 
                               data=json_input)
        logging.info(f'result: {result}')
        return result.json()['result']
    
def get_model():
    return ExternalModelPackage()


# Upload model

In [ ]:
project_id = 'tutorial'
model_id = 'external_model'
client.delete_model(project_id, model_id)
client.upload_model_package(model_dir, project_id, model_id)

# Test model

In [ ]:
client.run_model(project_id, model_id, df[0:20])